In [0]:
%run /Analysis/Authorization

In [0]:
#import required libraries
import datetime
import pyspark.sql.functions as f
import pyspark.sql.types
import pandas as pd

from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import Window
from pyspark.sql.functions import rank, min

In [0]:
#test check to access the container
dbutils.fs.ls("abfss://containerdatalake@samovieanalysis.dfs.core.windows.net")